In [ ]:
import mysql.connector

def add_to_db(word,word_ka,word_fr,word_ru,word_kr,word_jp,word_pronunciation,word_type,word_gender,word_tag):
        try:
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='RandoDumbai18',
                database='cosmolingo'
            )
            cursor = connection.cursor()
            query_check = "SELECT COUNT(*) FROM words WHERE word = %s"
            cursor.execute(query_check, (word,))
            result = cursor.fetchone()
            if result[0] == 0:
                query_insert = "INSERT INTO words (word,word_ka,word_fr,word_ru,word_kr,word_jp,word_pronunciation,word_type,word_gender,word_tag) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(query_insert, (word,word_ka,word_fr,word_ru,word_kr,word_jp,word_pronunciation,word_type,word_gender,word_tag))
            connection.commit()
        except mysql.connector.Error as err:
            print(f"Error: {err}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

In [21]:
add_to_db("test")

(1,)


In [ ]:
#query = "ALTER TABLE words AUTO_INCREMENT = 1;"

with open('words.txt') as f:
    words = f.read().split('\n')
    for word in words:
        word_meta = word.split(':')[0]
        if ';' in word_meta:
            word_tag = word_meta.split(';')[1]
            word_meta = word_meta.split(';')[0]
        else:
            word_tag = None
        if '-' not in word_meta:
            word_type = word_meta
            word_gender = None
        else:
            word_type = word_meta.split('-')[0]
            word_gender = word_meta.split('-')[1]
        word_translations = word.split(':')[1:]
        if '(' in word_translations[2]:
            word_pronunciation = word_translations[2].split('(')[1].split(')')[0]
            word_translations[2] = word_translations[2].split('(')[0]
        else:
            word_pronunciation = None
        word_ka = word_translations[0]
        word_en = word_translations[1]
        word_translations[0] = word_en
        word_translations[1] = word_ka
        
        add_to_db(*word_translations,word_pronunciation,word_type,word_gender,word_tag)